In [2]:
using Pkg; Pkg.add("Suppressor"); using Suppressor: @suppress;
@suppress Pkg.add(url="https://github.com/JuDO-dev/AirBorne.jl#dev")
@suppress Pkg.add(["Dates","Plots","DataFrames"])
@info "Dependencies added"
using AirBorne.ETL.YFinance: get_chart_data, parse_intraday_raw_data
using Dates: DateTime,datetime2unix

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


  No Changes to `~/Documents/uni-4/FYP/fyp_repo/Project.toml`
  No Changes to `~/Documents/uni-4/FYP/fyp_repo/Manifest.toml`


┌ Info: Dependencies added
└ @ Main /Users/sherifagbabiaka/Documents/uni-4/FYP/fyp_repo/duration_mpc.ipynb:4


In [3]:
using DataFrames
#input dates as "yyyy-mm-dd"
function get_and_process_data(tickers::Vector{String}, start_date::DateTime, end_date::DateTime, freq::String)
    unix(x) = string(round(Int, datetime2unix(DateTime(x))))
    function get_data()
        df = DataFrames.DataFrame()
        for ticker in tickers
            df = DataFrames.vcat(
                df, parse_intraday_raw_data(get_chart_data(ticker, unix(start_date), unix(end_date), freq))
            )
        end
        return df
    end
    df = get_data()
    processed_data = Dict()
    for ticker in tickers
        tmp = df[df[!, "symbol"] .== ticker, :]
        weekly = tmp
        insertcols!(weekly, "t+1 close" => vcat(weekly[!, "close"][2:end], missing))
        insertcols!(weekly, "t+2 close" => vcat(weekly[!, "close"][3:end], [missing, missing]))
        insertcols!(weekly, "t+3 close" => vcat(weekly[!, "close"][4:end], [missing, missing, missing]))
        insertcols!(weekly, "t+4 close" => vcat(weekly[!, "close"][5:end], [missing, missing, missing, missing]))
        dropmissing!(weekly)
        insertcols!(weekly, "1" => weekly[!, "t+1 close"] ./ weekly[!, "close"] .- 1)
        insertcols!(weekly, "2" => (weekly[!, "t+2 close"] ./ weekly[!, "close"]).^(1/2) .- 1)
        insertcols!(weekly, "3" => (weekly[!, "t+3 close"] ./ weekly[!, "close"]).^(1/3) .- 1)
        insertcols!(weekly, "4" => (weekly[!, "t+4 close"] ./ weekly[!, "close"]).^(1/4) .- 1)
        #insertcols!(weekly, "max returns" => maximum(weekly[!, ["t+1 returns", "t+2 returns", "t+3 returns", "t+4 returns"]], dims=2))
        # maximum((weekly[!, ["t+1 returns", "t+2 returns", "t+3 returns", "t+4 returns"]], dims=2))
        processed = combine(weekly, 1:6,11,17:20, AsTable(["1", "2", "3", "4"]) .=> ByRow.(findmax) => ["max returns", "duration"] )
        conversion_func(x) = parse(Int, string(x))
        processed_data[ticker] = combine(processed, :, "duration" .=> ByRow.(conversion_func) => "duration")
    end
    return processed_data
end

get_and_process_data (generic function with 1 method)

In [7]:
using DirectSearch

# series = get_and_process_data(["GOOG", "AAPL"], DateTime("2023-11-25"), DateTime("2024-01-15"), "5m")
series = get_and_process_data(["GOOG", "AAPL"], DateTime("2022-01-01"), DateTime("2023-01-01"), "1d")

#print(series["GOOG"])
total = nrow(series["GOOG"])
println("total = ", total)
trade = 0.01
dur = 1
portfolio = [0.5, 0.5]
println(series["GOOG"][:, :])

value = 1

for i = 1:total
    if dur == 1
        println("reoptimizing, week = ", i)
        #COVARIANCE MEASURE (WRT TIME?)
        #DIFFERENT COST MEASURE
        obj(x) = -x[1]*series["GOOG"][i, :]["max returns"] - x[2]*series["AAPL"][i, :]["max returns"] + trade * (abs(x[1] - portfolio[1]) + abs(x[2] - portfolio[2]))
        p = DSProblem(2, objective=obj, lower=[0.,0.], upper=[1.,1.])
        valcond(x) = x[1] + x[2] <= 1
        buyonly(x) = x[1] >=0 && x[2] >= 0
        duration(x) = x[1]*series["GOOG"][i, :]["duration"] + x[2]*series["AAPL"][i, :]["duration"] >= 1
        AddExtremeConstraint(p, buyonly)
        AddExtremeConstraint(p, valcond)
        SetGranularity(p, [0.1, 0.1])
        SetInitialPoint(p, [0.5, 0.5])
        @suppress Optimize!(p)
        dur = round(Int, p.x[1] * series["GOOG"][i, :]["duration"] + p.x[2] * series["AAPL"][i, :]["duration"])
        if dur == 0
            dur = 1
        end
        println("duration = ", dur)
        returns = ((series["GOOG"][i, :]["1"]) * p.x[1] + series["AAPL"][i, :]["1"] * p.x[2])
        value *= 1 + returns
        value -= trade * (abs(p.x[1] - portfolio[1]) + abs(p.x[2] - portfolio[2]))
        portfolio = p.x
    else
        returns = (series["GOOG"][i, :]["1"] * portfolio[1] + series["AAPL"][i, :]["1"] * portfolio[2])
        value *= 1 + returns
        dur -= 1
    end
    println("value = ", value)

end

println("final value = ", value)





# cov_matrix = cov(google[!, "close"], apple[!, "close"])
# print(cov_matrix)

# MONEY UNDER MATTRESS
# INTEREST RATE/SAVINGS/FIXED TERM
# BONDS
# S&P